# Original Linear Programming Model

In [ ]:
!pip install pulp
from pulp import LpMaximize, LpProblem, LpVariable
from pulp import LpMaximize, LpProblem, LpVariable, LpStatus, value
import pulp
from pulp import LpMaximize, LpProblem, LpVariable, LpStatus, value
prob = LpProblem("Farm_Optimization", LpMaximize)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 31.6 MB/s eta 0:00:00


In [ ]:
# Decision Variables
x1 = LpVariable("x1", 0)  # Number of heifers to be sold
x2 = LpVariable("x2", 0)  # Number of dairy cows to be sold
y1 = LpVariable("y1", 0)  # Number of heifers to be raised until age two to become dairy cows
y2 = LpVariable("y2", 0)  # Number of dairy cows to be kept for milk production
s1 = LpVariable("s1", 5)  # Tons of grain to be grown on the farm
s2 = LpVariable("s2", 15)  # Tons of sugar beet to be grown on the farm
w = LpVariable("w", 0)    # total workforce cost
E = LpVariable("E", 0)    # equipment cost
G1 = LpVariable("G1", 0)  # Tons of grain consumed by a dairy cow in a year
SB1 = LpVariable("SB1", 0)  # Tons of sugar beet consumed by a dairy cow in a year
Gr_buy = LpVariable("Gr_buy", 0)  # Tons of buy in grains
Gr_sell = LpVariable("Gr_sell", 0)  # Tons of sell out grains
SB_buy = LpVariable("SB_buy", 0)  # Tons of buy in sugar beet
SB_sell = LpVariable("SB_sell", 0)  # Tons of sell out sugar beet
U_cost = LpVariable("U_cost", 0)  # annual cost for Utility


In [ ]:
# Parameters
SP_fbaby_cows = 2800  # Selling price for one heifer
SP_dairy_cow = 2820  # Selling price for one dairy cow
B_seed_sugar_beet = 300  # Buy-in price for sugar beet seed per bag/acre
B_seed_grain = 100  # Buy-in price for grain seed per bag/acre
S_price_sugar_beet = 60  # Sell-out price for a ton of sugar beet
S_price_grain = 280  # Sell-out price for a ton of grains
Acre1 = 300  # Farm size in acres
B = 11000  # Budget for purchasing grain and sugar beets
G1 = 3  # Tons of grain consumed by a dairy cow in a year
SB1 = 2  # Tons of sugar beet consumed by a dairy cow in a year

## Objective Function

In [ ]:
# Objective Function
Total_Revenue_Female = x1 * SP_fbaby_cows
Total_Revenue_Dairy_Cow = x2 * SP_dairy_cow
Total_Revenue_Grain = Gr_sell * S_price_grain
Total_Revenue_Sugar_Beet = SB_sell * S_price_sugar_beet

Total_Grain_Cost = Gr_buy * B_seed_grain
Total_Sugar_Beet_Cost = SB_buy * B_seed_sugar_beet
Total_Operation_Cost = E + U_cost + w

# Maximize Profit
profit = (Total_Revenue_Female + Total_Revenue_Dairy_Cow + Total_Revenue_Grain + Total_Revenue_Sugar_Beet) - (Total_Grain_Cost + Total_Sugar_Beet_Cost + Total_Operation_Cost)
prob += profit, 'Total Profit'

## Constraints

In [ ]:
prob += (x1 <= y1, "Female Baby Cows Sold Constraint")
prob += (x2 <= y2, "Dairy Cows Sold Constraint")
prob += (x1 + x2 + y1 + y2 <= 110, "Cow Housing Constraint")
prob += (G1 <= Gr_buy - Gr_sell, "Grain Consumption Constraint")
prob += (SB1 <= SB_buy - SB_sell, "Sugar Beet Consumption Constraint")
prob += (Gr_sell <= s1 - G1, "Grain Sale Constraint")
prob += (SB_sell <= s2 - SB1, "Sugar Beet Sale Constraint")
prob += (U_cost == 0.04 * profit, "Utility Cost Constraint")
prob += (w == 0.08 * profit, "Workforce Cost Constraint")
prob += (Gr_buy >= G1 * y2 + G1 * y1, "Grain Buying Constraint")
prob += (SB_buy >= SB1 * y2 + SB1 * y1, "Sugar Beet Buying Constraint")
prob += (E == 0.15 * Total_Operation_Cost, "Equipment Cost Constraint")
prob += (Total_Grain_Cost + Total_Sugar_Beet_Cost <= B, "Budget Constraint")
prob += (x1 >= 5, "Min Number of Female Baby Cows to be Sold")
prob += (y1 >= 5, "Min Number of Female Baby Cows to be Raised")

In [ ]:
# Solve the Problem
prob.solve()
print("Status:", LpStatus[prob.status])
if prob.status > 0:
    print("Total Profit = ", value(prob.objective))
    for v in prob.variables():
        print(v.name, "=", v.varValue)
else:
    print("No solution found!")

Status: Optimal
Total Profit =  29916.494934
E = 633.52577
Gr_buy = 36.666667
Gr_sell = 33.666667
SB_buy = 24.444444
SB_sell = 22.444444
U_cost = 1196.6598
s1 = 36.666667
s2 = 24.444444
w = 2393.3196
x1 = 5.0
x2 = 7.2222222
y1 = 5.0
y2 = 7.2222222


## Sensitivity Analysis
### Shadow Prices of Each Constraint

In [ ]:
for name, constraint in prob.constraints.items():
    print(name, ":", constraint.pi)

Female_Baby_Cows_Sold_Constraint : 2471.134
Dairy_Cows_Sold_Constraint : 2471.134
Cow_Housing_Constraint : -0.0
Grain_Consumption_Constraint : -245.36082
Sugar_Beet_Consumption_Constraint : -52.57732
Grain_Sale_Constraint : -0.0
Sugar_Beet_Sale_Constraint : -0.0
Utility_Cost_Constraint : -1.0309278
Workforce_Cost_Constraint : -1.0309278
Grain_Buying_Constraint : -122.68041
Sugar_Beet_Buying_Constraint : -1051.5464
Equipment_Cost_Constraint : -1.0309278
Budget_Constraint : 2.8041237
Min_Number_of_Female_Baby_Cows_to_be_Sold : -17.525773
Min_Number_of_Female_Baby_Cows_to_be_Raised : -0.0


### Allowable Increase/Decrease for each constraint

In [ ]:
# Given Parameters
# Prices and Costs
selling_price_heifer = 2800  # $ per heifer
selling_price_dairy_cow = 3000  # $ per dairy cow
buyin_price_sugar_beet_seed = 300  # $ per bag/acre
buyin_price_grain_seed = 100  # $ per bag/acre
sellout_price_sugar_beet = 60  # $ per ton
sellout_price_grain = 280  # $ per ton
# Farm size
Acre1 = 300  # in acres
# Budget for purchasing grain and sugar beets
B = 11000  # $
# Consumption rates
G1 = 3  # Tons of grain consumed by a dairy cow in a year
SB1 = 2  # Tons of sugar beet consumed by a dairy cow in a year
# These parameters are defined for clarity and completeness
selling_price_heifer, selling_price_dairy_cow, buyin_price_sugar_beet_seed, buyin_price_grain_seed, sellout_price_sugar_beet, sellout_price_grain, Acre1, B, G1, SB1

(2800, 3000, 300, 100, 60, 280, 300, 11000, 3, 2)

In [ ]:
from scipy.optimize import linprog
import numpy as np
c = [
    -selling_price_heifer,  # x1
    -selling_price_dairy_cow,  # x2
    0,  # y1
    0,  # y2
    0,  # s1
    0,  # s2
    0.1,  # w
    0,  # E
    0,  # Gr_buy
    sellout_price_grain,  # Gr_sell
    0,  # SB_buy
    sellout_price_sugar_beet,  # SB_sell
    0.04,  # U_cost
]
A = [
    # Heifers Sold Constraint
    [1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],

    # Dairy Cows Sold Constraint
    [0, 1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0],

    # Cow Housing Constraint
    [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],

    # Grain Consumption Constraint
    [0, 0, 0, 0, 0, 0, 0, 0, 1, -1, 0, 0, 0],

    # Sugar Beet Consumption Constraint
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, 0],

    # Grain Sale Constraint
    [0, 0, 0, 0, 1, 0, 0, 0, 0, -1, 0, 0, 0],

    # Sugar Beet Sale Constraint
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, -1, 0],

    # Grain Buying Constraint (modified to <=)
    [0, 0, -G1, -G1, 0, 0, 0, 0, -1, 0, 0, 0, 0],

    # Sugar Beet Buying Constraint (modified to <=)
    [0, 0, -SB1, -SB1, 0, 0, 0, 0, 0, 0, -1, 0, 0],

    # Budget Constraint
    [0, 0, 0, 0, 0, 0, 0, 0, buyin_price_grain_seed, 0, buyin_price_sugar_beet_seed, 0, 0],
]

# Upper Bounds for Inequality Constraints (b)
b = [
    0,  # Heifers Sold Constraint
    0,  # Dairy Cows Sold Constraint
    110,  # Cow Housing Constraint
    G1,  # Grain Consumption Constraint
    SB1,  # Sugar Beet Consumption Constraint
    0,  # Grain Sale Constraint
    0,  # Sugar Beet Sale Constraint
    0,  # Grain Buying Constraint
    0,  # Sugar Beet Buying Constraint
    B,   # Budget Constraint
]

# Bounds for Decision Variables (min, max) pairs
x_bounds = [
    (5, None),  # x1: Min Number of Heifers to be Sold
    (0, None),  # x2
    (5, None),  # y1: Min Number of Heifers to be Raised
    (0, None),  # y2
    (0, None),  # s1
    (0, None),  # s2
    (0, None),  # w
    (0, None),  # E
    (0, None),  # Gr_buy
    (0, None),  # Gr_sell
    (0, None),  # SB_buy
    (0, None),  # SB_sell
    (0, None),  # U_cost
]
# Solve the Linear Programming Problem
result = linprog(c, A_ub=A, b_ub=b, bounds=x_bounds, method='highs')
result

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -164000.0
              x: [ 5.000e+00  5.000e+01 ...  0.000e+00  0.000e+00]
            nit: 0
          lower:  residual: [ 0.000e+00  5.000e+01 ...  0.000e+00
                              0.000e+00]
                 marginals: [ 2.000e+02  0.000e+00 ...  6.000e+01
                              4.000e-02]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00  0.000e+00  3.000e+00
                              2.000e+00  0.000e+00  0.000e+00  1.650e+02
                              1.100e+02  1.100e+04]
                 marginals: [-1.500e+03 -1.500e+03 -1.500e+03 -0.000e+00
    

In [ ]:
# Sensitivity Analysis: Allowable Increase for each constraint
perturb = 0.01
original_profit = -result.fun
# Initialize a list to store the Allowable Increase for each constraint
allowable_increase = []
# Perform sensitivity analysis
for i in range(len(b)):
    b_perturbed = b.copy()
    b_perturbed[i] += perturb * b[i]
    result_perturbed = linprog(c, A_ub=A, b_ub=b_perturbed, bounds=x_bounds, method='highs')
    change_in_profit = -result_perturbed.fun - original_profit
    allowable_increase.append(change_in_profit)
allowable_increase

[0.0, 0.0, 1650.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [ ]:
# Sensitivity Analysis: Allowable Decrease for each constraint
# Initialize a list to store the Allowable Decrease for each constraint
allowable_decrease = []
# Perform sensitivity analysis
for i in range(len(b)):
    b_perturbed = b.copy()
    b_perturbed[i] -= perturb * b[i]
    result_perturbed = linprog(c, A_ub=A, b_ub=b_perturbed, bounds=x_bounds, method='highs')
    change_in_profit = -result_perturbed.fun - original_profit
    allowable_decrease.append(change_in_profit)

allowable_decrease

import pandas as pd
# List of constraint names
constraint_names = [
    "Female Baby Cows Sold Constraint",
    "Dairy Cows Sold Constraint",
    "Cow Housing Constraint",
    "Grain Sale Constraint",
    "Sugar Beet Sale Constraint",
    "Grain Buying Constraint",
    "Sugar Beet Buying Constraint",
    "Utility Cost Constraint",
    "Workforce Cost Constraint",
    "Equipment Cost Constraint"
]
# Create a DataFrame for the table
sensitivity_table_constraints = pd.DataFrame({
    "Constraint": constraint_names,
    "Allowable Increase": allowable_increase,
    "Allowable Decrease": allowable_decrease
})
# Display the table
sensitivity_table_constraints

,Constraint,Allowable Increase,Allowable Decrease
0,Female Baby Cows Sold Constraint,0.0,0.0
1,Dairy Cows Sold Constraint,0.0,0.0
2,Cow Housing Constraint,1650.0,-1650.0
3,Grain Sale Constraint,0.0,0.0
4,Sugar Beet Sale Constraint,0.0,0.0
5,Grain Buying Constraint,0.0,0.0
6,Sugar Beet Buying Constraint,0.0,0.0
7,Utility Cost Constraint,0.0,0.0
8,Workforce Cost Constraint,0.0,0.0
9,Equipment Cost Constraint,0.0,0.0


The allowable increase column says that we can increase the cow housing constraint by 1650 before the shadow price changes.

In [ ]:
# Sensitivity Analysis: Allowable Increase for Decision Variables
perturb_obj = 0.01
original_profit_obj = -result.fun
allowable_increase_obj = []

# Perform sensitivity analysis
for i in range(len(c)):
    # Perturb the objective coefficient by a small amount (increase)
    c_perturbed = c.copy()
    c_perturbed[i] += perturb_obj * c[i]
    result_perturbed_obj = linprog(c_perturbed, A_ub=A, b_ub=b, bounds=x_bounds, method='highs')
    change_in_profit_obj = -result_perturbed_obj.fun - original_profit_obj
    allowable_increase_obj.append(change_in_profit_obj)

allowable_increase_obj

allowable_decrease_obj = []
# Perform sensitivity analysis
for i in range(len(c)):

    c_perturbed = c.copy()
    c_perturbed[i] -= perturb_obj * c[i]
    result_perturbed_obj = linprog(c_perturbed, A_ub=A, b_ub=b, bounds=x_bounds, method='highs')
    change_in_profit_obj = -result_perturbed_obj.fun - original_profit_obj
    allowable_decrease_obj.append(change_in_profit_obj)

allowable_decrease_obj

import pandas as pd
decision_variables = [
    "x1 (Female Baby Cows Sold)",
    "x2 (Dairy Cows Sold)",
    "y1 (Female Baby Cows Raised)",
    "y2 (Dairy Cows for Milk)",
    "s1 (Grain Grown)",
    "s2 (Sugar Beet Grown)",
    "w (Workforce Cost)",
    "E (Equipment Cost)",
    "Gr_buy (Buy in Grains)",
    "Gr_sell (Sell out Grains)",
    "SB_buy (Buy in Sugar Beet)",
    "SB_sell (Sell out Sugar Beet)",
    "U_cost (Utility Cost)"
]

sensitivity_table_obj = pd.DataFrame({
    "Decision Variable": decision_variables,
    "Allowable Increase": allowable_increase_obj,
    "Allowable Decrease": allowable_decrease_obj

})
# Display the table
sensitivity_table_obj

,Decision Variable,Allowable Increase,Allowable Decrease
0,x1 (Female Baby Cows Sold),140.0,-140.0
1,x2 (Dairy Cows Sold),1500.0,-1500.0
2,y1 (Female Baby Cows Raised),0.0,0.0
3,y2 (Dairy Cows for Milk),0.0,0.0
4,s1 (Grain Grown),0.0,0.0
5,s2 (Sugar Beet Grown),0.0,0.0
6,w (Workforce Cost),0.0,0.0
7,E (Equipment Cost),0.0,0.0
8,Gr_buy (Buy in Grains),0.0,0.0
9,Gr_sell (Sell out Grains),0.0,0.0


The profit from selling one female baby cow can increase 140 dollars or decrease by 140 dollars without changing the optimum.

The profit from selling one dairy cow can increase 1500 dollars or decrease by 1500 dollars without changing the optimum.

# Worst Scenario




In [ ]:
!pip install pulp
from pulp import LpMaximize, LpProblem, LpVariable
from pulp import LpMaximize, LpProblem, LpVariable, LpStatus, value
import pulp
from pulp import LpMaximize, LpProblem, LpVariable, LpStatus, value

# Create the 'Farm_Optimization' problem
prob = LpProblem("Farm_Optimization", LpMaximize)


In [ ]:
# Decision Variables
x1 = LpVariable("x1", 0)  # Number of female baby cows to be sold
x2 = LpVariable("x2", 0)  # Number of dairy cows to be sold
y1 = LpVariable("y1", 0)  # Number of female baby cows to be raised until age two to become dairy cows
y2 = LpVariable("y2", 0)  # Number of dairy cows to be kept for milk production
s1 = LpVariable("s1", 5)  # Tons of grain to be grown on the farm
s2 = LpVariable("s2", 15)  # Tons of sugar beet to be grown on the farm
w = LpVariable("w", 0)    # total workforce cost
E = LpVariable("E", 0)    # equipment cost
G1 = LpVariable("G1", 0)  # Tons of grain consumed by a dairy cow in a year
SB1 = LpVariable("SB1", 0)  # Tons of sugar beet consumed by a dairy cow in a year
Gr_buy = LpVariable("Gr_buy", 0)  # Tons of buy in grains
Gr_sell = LpVariable("Gr_sell", 0)  # Tons of sell out grains
SB_buy = LpVariable("SB_buy", 0)  # Tons of buy in sugar beet
SB_sell = LpVariable("SB_sell", 0)  # Tons of sell out sugar beet
U_cost = LpVariable("U_cost", 0)  # annual cost for Utility


In [ ]:
# Parameters
SP_fbaby_cow = 2500  # Selling price for one female baby cow
SP_dairy_cow = 1000  # Selling price for one dairy cow
B_seed_sugar_beet = 300  # Buy-in price for sugar beet seed per bag/acre
B_seed_grain = 100  # Buy-in price for grain seed per bag/acre
S_price_sugar_beet = 60  # Sell-out price for a ton of sugar beet
S_price_grain = 280  # Sell-out price for a ton of grains
Acre1 = 300  # Farm size in acres
B = 8000  # Budget for purchasing grain and sugar beets
G1 = 3  # Tons of grain consumed by a dairy cow in a year
SB1 = 2  # Tons of sugar beet consumed by a dairy cow in a year

In [ ]:
# Objective Function
Total_Revenue_Heifer = x1 * SP_fbaby_cow
Total_Revenue_Dairy_Cow = x2 * SP_dairy_cow
Total_Revenue_Grain = Gr_sell * S_price_grain
Total_Revenue_Sugar_Beet = SB_sell * S_price_sugar_beet

Total_Grain_Cost = Gr_buy * B_seed_grain
Total_Sugar_Beet_Cost = SB_buy * B_seed_sugar_beet
Total_Operation_Cost = E + U_cost + w

# Maximize Profit
profit = (Total_Revenue_Heifer + Total_Revenue_Dairy_Cow + Total_Revenue_Grain + Total_Revenue_Sugar_Beet) - (Total_Grain_Cost + Total_Sugar_Beet_Cost + Total_Operation_Cost)
prob += profit, 'Total Profit'


In [ ]:
# Constraints
prob += (x1 <= y1, "Female Baby Cows Sold Constraint")
prob += (x2 <= y2, "Dairy Cows Sold Constraint")
prob += (x1 + x2 + y1 + y2 <= 90, "Cow Housing Constraint")
prob += (G1 <= Gr_buy - Gr_sell, "Grain Consumption Constraint")
prob += (SB1 <= SB_buy - SB_sell, "Sugar Beet Consumption Constraint")
prob += (Gr_sell <= s1 - G1, "Grain Sale Constraint")
prob += (SB_sell <= s2 - SB1, "Sugar Beet Sale Constraint")
prob += (U_cost == 0.06 * profit, "Utility Cost Constraint")
prob += (w == 0.12 * profit, "Workforce Cost Constraint")
prob += (Gr_buy >= G1 * y2 + G1 * y1, "Grain Buying Constraint")
prob += (SB_buy >= SB1 * y2 + SB1 * y1, "Sugar Beet Buying Constraint")
prob += (E == 0.20 * Total_Operation_Cost, "Equipment Cost Constraint")
prob += (Total_Grain_Cost + Total_Sugar_Beet_Cost <= B, "Budget Constraint")
prob += (x1 >= 5, "Min Number of Female Baby Cows to be Sold")
prob += (y1 >= 7, "Min Number of Female Baby Cows to be Raised")

In [ ]:
# Solve the Problem
prob.solve()

#prob.solve(pulp.COIN_CMD())
print("Status:", LpStatus[prob.status])


if prob.status > 0:  # Ensure that the solver found a solution
    print("Total Profit = ", value(prob.objective))
    for v in prob.variables():
        print(v.name, "=", v.varValue)
else:
    print("No solution found!")

Status: Optimal
Total Profit =  17792.290329999996
E = 800.65306
Gr_buy = 26.666667
Gr_sell = 23.666667
SB_buy = 17.777778
SB_sell = 15.777778
U_cost = 1067.5374
s1 = 26.666667
s2 = 17.777778
w = 2135.0748
x1 = 8.8888889
x2 = 0.0
y1 = 8.8888889
y2 = 0.0


In [ ]:
# Print the Shadow Prices of Each Constraint
for name, constraint in prob.constraints.items():
    print(name, ":", constraint.pi)

Female_Baby_Cows_Sold_Constraint : 2040.8163
Dairy_Cows_Sold_Constraint : 2040.8163
Cow_Housing_Constraint : -0.0
Grain_Consumption_Constraint : -228.57143
Sugar_Beet_Consumption_Constraint : -48.979592
Grain_Sale_Constraint : -0.0
Sugar_Beet_Sale_Constraint : -0.0
Utility_Cost_Constraint : -1.0204082
Workforce_Cost_Constraint : -1.0204082
Grain_Buying_Constraint : -85.260771
Sugar_Beet_Buying_Constraint : -892.51701
Equipment_Cost_Constraint : -1.0204082
Budget_Constraint : 2.3219955
Min_Number_of_Female_Baby_Cows_to_be_Sold : -0.0
Min_Number_of_Female_Baby_Cows_to_be_Raised : -0.0


## Sensitivity Analysis
### Shadow Prices of Each Constraint

In [ ]:
for name, constraint in prob.constraints.items():
    print(name, ":", constraint.pi)

Female_Baby_Cows_Sold_Constraint : 2040.8163
Dairy_Cows_Sold_Constraint : 2040.8163
Cow_Housing_Constraint : -0.0
Grain_Consumption_Constraint : -228.57143
Sugar_Beet_Consumption_Constraint : -48.979592
Grain_Sale_Constraint : -0.0
Sugar_Beet_Sale_Constraint : -0.0
Utility_Cost_Constraint : -1.0204082
Workforce_Cost_Constraint : -1.0204082
Grain_Buying_Constraint : -85.260771
Sugar_Beet_Buying_Constraint : -892.51701
Equipment_Cost_Constraint : -1.0204082
Budget_Constraint : 2.3219955
Min_Number_of_Female_Baby_Cows_to_be_Sold : -0.0
Min_Number_of_Female_Baby_Cows_to_be_Raised : -0.0


### Allowable Increase/Decrease for each constraint

In [ ]:
# Prices and Costs
selling_price_heifer = 2500  # $ per heifer
selling_price_dairy_cow = 1000  # $ per dairy cow
buyin_price_sugar_beet_seed = 300  # $ per bag/acre
buyin_price_grain_seed = 100  # $ per bag/acre
sellout_price_sugar_beet = 60  # $ per ton
sellout_price_grain = 280  # $ per ton
# Farm size
Acre1 = 300  # in acres
# Budget for purchasing grain and sugar beets
B = 8000  # $
# Consumption rates
G1 = 3  # Tons of grain consumed by a dairy cow in a year
SB1 = 2  # Tons of sugar beet consumed by a dairy cow in a year
# These parameters are defined for clarity and completeness
selling_price_heifer, selling_price_dairy_cow, buyin_price_sugar_beet_seed, buyin_price_grain_seed, sellout_price_sugar_beet, sellout_price_grain, Acre1, B, G1, SB1

(2500, 1000, 300, 100, 60, 280, 300, 8000, 3, 2)

In [ ]:
from scipy.optimize import linprog
import numpy as np
c = [
    -selling_price_heifer,  # x1
    -selling_price_dairy_cow,  # x2
    0,  # y1
    0,  # y2
    0,  # s1
    0,  # s2
    0.1,  # w
    0,  # E
    0,  # Gr_buy
    sellout_price_grain,  # Gr_sell
    0,  # SB_buy
    sellout_price_sugar_beet,  # SB_sell
    0.04,  # U_cost
]
A = [
    # Heifers Sold Constraint
    [1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],

    # Dairy Cows Sold Constraint
    [0, 1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0],

    # Cow Housing Constraint
    [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],

    # Grain Consumption Constraint
    [0, 0, 0, 0, 0, 0, 0, 0, 1, -1, 0, 0, 0],

    # Sugar Beet Consumption Constraint
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, 0],

    # Grain Sale Constraint
    [0, 0, 0, 0, 1, 0, 0, 0, 0, -1, 0, 0, 0],

    # Sugar Beet Sale Constraint
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, -1, 0],

    # Grain Buying Constraint (modified to <=)
    [0, 0, -G1, -G1, 0, 0, 0, 0, -1, 0, 0, 0, 0],

    # Sugar Beet Buying Constraint (modified to <=)
    [0, 0, -SB1, -SB1, 0, 0, 0, 0, 0, 0, -1, 0, 0],

    # Budget Constraint
    [0, 0, 0, 0, 0, 0, 0, 0, buyin_price_grain_seed, 0, buyin_price_sugar_beet_seed, 0, 0],
]
# Upper Bounds for Inequality Constraints (b)
b = [
    0,  # Heifers Sold Constraint
    0,  # Dairy Cows Sold Constraint
    110,  # Cow Housing Constraint
    G1,  # Grain Consumption Constraint
    SB1,  # Sugar Beet Consumption Constraint
    0,  # Grain Sale Constraint
    0,  # Sugar Beet Sale Constraint
    0,  # Grain Buying Constraint
    0,  # Sugar Beet Buying Constraint
    B,   # Budget Constraint
]
# Bounds for Decision Variables (min, max) pairs
x_bounds = [
    (5, None),  # x1: Min Number of Heifers to be Sold
    (0, None),  # x2
    (5, None),  # y1: Min Number of Heifers to be Raised
    (0, None),  # y2
    (0, None),  # s1
    (0, None),  # s2
    (0, None),  # w
    (0, None),  # E
    (0, None),  # Gr_buy
    (0, None),  # Gr_sell
    (0, None),  # SB_buy
    (0, None),  # SB_sell
    (0, None),  # U_cost
]
# Solve the Linear Programming Problem
result = linprog(c, A_ub=A, b_ub=b, bounds=x_bounds, method='highs')
result

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -137500.0
              x: [ 5.500e+01  0.000e+00 ...  0.000e+00  0.000e+00]
            nit: 0
          lower:  residual: [ 5.000e+01  0.000e+00 ...  0.000e+00
                              0.000e+00]
                 marginals: [ 0.000e+00  2.500e+02 ...  6.000e+01
                              4.000e-02]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00  0.000e+00  3.000e+00
                              2.000e+00  0.000e+00  0.000e+00  1.650e+02
                              1.100e+02  8.000e+03]
                 marginals: [-1.250e+03 -0.000e+00 -1.250e+03 -0.000e+00
    

In [ ]:
# Sensitivity Analysis: Allowable Increase for each constraint
perturb = 0.01
original_profit = -result.fun
# Initialize a list to store the Allowable Increase for each constraint
allowable_increase = []
# Perform sensitivity analysis
for i in range(len(b)):
    b_perturbed = b.copy()
    b_perturbed[i] += perturb * b[i]
    result_perturbed = linprog(c, A_ub=A, b_ub=b_perturbed, bounds=x_bounds, method='highs')
    change_in_profit = -result_perturbed.fun - original_profit
    allowable_increase.append(change_in_profit)
allowable_increase

[0.0, 0.0, 1375.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [ ]:
# Sensitivity Analysis: Allowable Decrease for each constraint
# Initialize a list to store the Allowable Decrease for each constraint
allowable_decrease = []

# Perform sensitivity analysis
for i in range(len(b)):
    b_perturbed = b.copy()
    b_perturbed[i] -= perturb * b[i]
    result_perturbed = linprog(c, A_ub=A, b_ub=b_perturbed, bounds=x_bounds, method='highs')
    change_in_profit = -result_perturbed.fun - original_profit
    allowable_decrease.append(change_in_profit)

allowable_decrease
import pandas as pd
# List of constraint names
constraint_names = [
    "Female Baby Cows Sold Constraint",
    "Dairy Cows Sold Constraint",
    "Cow Housing Constraint",
    "Grain Sale Constraint",
    "Sugar Beet Sale Constraint",
    "Grain Buying Constraint",
    "Sugar Beet Buying Constraint",
    "Utility Cost Constraint",
    "Workforce Cost Constraint",
    "Equipment Cost Constraint"
]
# Create a DataFrame for the table
sensitivity_table_constraints = pd.DataFrame({
    "Constraint": constraint_names,
    "Allowable Increase": allowable_increase,
    "Allowable Decrease": allowable_decrease
})
# Display the table
sensitivity_table_constraints

,Constraint,Allowable Increase,Allowable Decrease
0,Female Baby Cows Sold Constraint,0.0,0.0
1,Dairy Cows Sold Constraint,0.0,0.0
2,Cow Housing Constraint,1375.0,-1375.0
3,Grain Sale Constraint,0.0,0.0
4,Sugar Beet Sale Constraint,0.0,0.0
5,Grain Buying Constraint,0.0,0.0
6,Sugar Beet Buying Constraint,0.0,0.0
7,Utility Cost Constraint,0.0,0.0
8,Workforce Cost Constraint,0.0,0.0
9,Equipment Cost Constraint,0.0,0.0


The allowable increase column says that we can increase the cow housing constraint by 1375 before the shadow price changes.

In [ ]:
# Sensitivity Analysis: Allowable Increase for Decision Variables
# Small perturbation value (1%)
perturb_obj = 0.01

original_profit_obj = -result.fun
# Initialize a list to store the Allowable Increase for each decision variable
allowable_increase_obj = []

for i in range(len(c)):
    # Perturb the objective coefficient by a small amount (increase)
    c_perturbed = c.copy()
    c_perturbed[i] += perturb_obj * c[i]
    result_perturbed_obj = linprog(c_perturbed, A_ub=A, b_ub=b, bounds=x_bounds, method='highs')
    change_in_profit_obj = -result_perturbed_obj.fun - original_profit_obj
    allowable_increase_obj.append(change_in_profit_obj)

allowable_increase_obj
allowable_decrease_obj = []

# Perform sensitivity analysis
for i in range(len(c)):

    c_perturbed = c.copy()
    c_perturbed[i] -= perturb_obj * c[i]
    result_perturbed_obj = linprog(c_perturbed, A_ub=A, b_ub=b, bounds=x_bounds, method='highs')
    change_in_profit_obj = -result_perturbed_obj.fun - original_profit_obj
    allowable_decrease_obj.append(change_in_profit_obj)

allowable_decrease_obj
import pandas as pd
decision_variables = [
    "x1 (Female Baby Cows Sold)",
    "x2 (Dairy Cows Sold)",
    "y1 (Female Baby Cows Raised)",
    "y2 (Dairy Cows for Milk)",
    "s1 (Grain Grown)",
    "s2 (Sugar Beet Grown)",
    "w (Workforce Cost)",
    "E (Equipment Cost)",
    "Gr_buy (Buy in Grains)",
    "Gr_sell (Sell out Grains)",
    "SB_buy (Buy in Sugar Beet)",
    "SB_sell (Sell out Sugar Beet)",
    "U_cost (Utility Cost)"
]

sensitivity_table_obj = pd.DataFrame({
    "Decision Variable": decision_variables,
    "Allowable Increase": allowable_increase_obj,
    "Allowable Decrease": allowable_decrease_obj,
})
# Display the table
sensitivity_table_obj

,Decision Variable,Allowable Increase,Allowable Decrease
0,x1 (Female Baby Cows Sold),1375.0,-1375.0
1,x2 (Dairy Cows Sold),0.0,0.0
2,y1 (Female Baby Cows Raised),0.0,0.0
3,y2 (Dairy Cows for Milk),0.0,0.0
4,s1 (Grain Grown),0.0,0.0
5,s2 (Sugar Beet Grown),0.0,0.0
6,w (Workforce Cost),0.0,0.0
7,E (Equipment Cost),0.0,0.0
8,Gr_buy (Buy in Grains),0.0,0.0
9,Gr_sell (Sell out Grains),0.0,0.0


The profit from selling one female baby cow can increase 1375 dollars or decrease by 1375 dollars without changing the optimum.

# Best Case Scenario

In [ ]:
!pip install pulp
from pulp import LpMaximize, LpProblem, LpVariable
from pulp import LpMaximize, LpProblem, LpVariable, LpStatus, value
import pulp
from pulp import LpMaximize, LpProblem, LpVariable, LpStatus, value

# Create the 'Farm_Optimization' problem
prob = LpProblem("Farm_Optimization", LpMaximize)

In [ ]:
# Decision Variables
x1 = LpVariable("x1", 0)  # Number of female baby cows to be sold
x2 = LpVariable("x2", 0)  # Number of dairy cows to be sold
y1 = LpVariable("y1", 0)  # Number of female baby cows to be raised until age two to become dairy cows
y2 = LpVariable("y2", 0)  # Number of dairy cows to be kept for milk production
s1 = LpVariable("s1", 5)  # Tons of grain to be grown on the farm
s2 = LpVariable("s2", 15)  # Tons of sugar beet to be grown on the farm
w = LpVariable("w", 0)    # total workforce cost
E = LpVariable("E", 0)    # equipment cost
G1 = LpVariable("G1", 0)  # Tons of grain consumed by a dairy cow in a year
SB1 = LpVariable("SB1", 0)  # Tons of sugar beet consumed by a dairy cow in a year
Gr_buy = LpVariable("Gr_buy", 0)  # Tons of buy in grains
Gr_sell = LpVariable("Gr_sell", 0)  # Tons of sell out grains
SB_buy = LpVariable("SB_buy", 0)  # Tons of buy in sugar beet
SB_sell = LpVariable("SB_sell", 0)  # Tons of sell out sugar beet
U_cost = LpVariable("U_cost", 0)  # annual cost for Utility


In [ ]:
# Parameters
SP_fbaby_cow = 3000 # Selling price for one female baby cow
SP_dairy_cow = 4000  # Selling price for one dairy cow
B_seed_sugar_beet = 350  # Buy-in price for sugar beet seed per bag/acre
B_seed_grain = 150  # Buy-in price for grain seed per bag/acre
S_price_sugar_beet = 80  # Sell-out price for a ton of sugar beet
S_price_grain = 300  # Sell-out price for a ton of grains
Acre1 = 300  # Farm size in acres
B = 12000  # Budget for purchasing grain and sugar beets
G1 = 3  # Tons of grain consumed by a dairy cow in a year
SB1 = 2  # Tons of sugar beet consumed by a dairy cow in a year

In [ ]:
# Objective Function
Total_Revenue_fbaby_cow = x1 * SP_fbaby_cow
Total_Revenue_Dairy_Cow = x2 * SP_dairy_cow
Total_Revenue_Grain = Gr_sell * S_price_grain
Total_Revenue_Sugar_Beet = SB_sell * S_price_sugar_beet

Total_Grain_Cost = Gr_buy * B_seed_grain
Total_Sugar_Beet_Cost = SB_buy * B_seed_sugar_beet
Total_Operation_Cost = E + U_cost + w

# Maximize Profit
profit = (Total_Revenue_fbaby_cow + Total_Revenue_Dairy_Cow + Total_Revenue_Grain + Total_Revenue_Sugar_Beet) - (Total_Grain_Cost + Total_Sugar_Beet_Cost + Total_Operation_Cost)
prob += profit, 'Total Profit'

In [ ]:
# Constraints
prob += (x1 <= y1, "Female Baby Cows Sold Constraint")
prob += (x2 <= y2, "Dairy Cows Sold Constraint")
prob += (x1 + x2 + y1 + y2 <= 150, "Cow Housing Constraint")
prob += (G1 <= Gr_buy - Gr_sell, "Grain Consumption Constraint")
prob += (SB1 <= SB_buy - SB_sell, "Sugar Beet Consumption Constraint")
prob += (Gr_sell <= s1 - G1, "Grain Sale Constraint")
prob += (SB_sell <= s2 - SB1, "Sugar Beet Sale Constraint")
prob += (U_cost == 0.02 * profit, "Utility Cost Constraint")
prob += (w == 0.06 * profit, "Workforce Cost Constraint")
prob += (Gr_buy >= G1 * y2 + G1 * y1, "Grain Buying Constraint")
prob += (SB_buy >= SB1 * y2 + SB1 * y1, "Sugar Beet Buying Constraint")
prob += (E == 0.05 * Total_Operation_Cost, "Equipment Cost Constraint")
prob += (Total_Grain_Cost + Total_Sugar_Beet_Cost <= B, "Budget Constraint")
prob += (x1 >= 3, "Min Number of Female Baby Cows to be Sold")
prob += (y1 >= 3, "Min Number of Female Baby Cows to be Raised")

In [ ]:
# Solve the Problem
prob.solve()

#prob.solve(pulp.COIN_CMD())
print("Status:", LpStatus[prob.status])
if prob.status > 0:  # Ensure that the solver found a solution
    print("Total Profit = ", value(prob.objective))
    for v in prob.variables():
        print(v.name, "=", v.varValue)
else:
    print("No solution found!")

Status: Optimal
Total Profit =  33886.40778
E = 142.67961
Gr_buy = 31.304348
Gr_sell = 28.304348
SB_buy = 20.869565
SB_sell = 18.869565
U_cost = 677.72816
s1 = 31.304348
s2 = 20.869565
w = 2033.1845
x1 = 3.0
x2 = 7.4347826
y1 = 3.0
y2 = 7.4347826


## Sensitivity Analysis

**Shadow Prices**

In [ ]:
# Print the Shadow Prices of Each Constraint
for name, constraint in prob.constraints.items():
    print(name, ":", constraint.pi)

Female_Baby_Cows_Sold_Constraint : 3689.3204
Dairy_Cows_Sold_Constraint : 3689.3204
Cow_Housing_Constraint : -0.0
Grain_Consumption_Constraint : -276.69903
Sugar_Beet_Consumption_Constraint : -73.786408
Grain_Sale_Constraint : -0.0
Sugar_Beet_Sale_Constraint : -0.0
Utility_Cost_Constraint : -0.97087379
Workforce_Cost_Constraint : -0.97087379
Grain_Buying_Constraint : -332.03883
Sugar_Beet_Buying_Constraint : -1346.6019
Equipment_Cost_Constraint : -0.97087379
Budget_Constraint : 3.1359223
Min_Number_of_Female_Baby_Cows_to_be_Sold : -922.3301
Min_Number_of_Female_Baby_Cows_to_be_Raised : -0.0


### Allowable Increase/Decrease for each Constraint

In [ ]:
# Prices and Costs
selling_price_fbaby_cow = 3000  # $ per female baby cow
selling_price_dairy_cow = 4000  # $ per dairy cow
buyin_price_sugar_beet_seed = 350  # $ per bag/acre
buyin_price_grain_seed = 150  # $ per bag/acre
sellout_price_sugar_beet = 80  # $ per ton
sellout_price_grain = 300  # $ per ton
# Farm size
Acre1 = 300  # in acres
# Budget for purchasing grain and sugar beets
B = 12000  # $
# Consumption rates
G1 = 3  # Tons of grain consumed by a dairy cow in a year
SB1 = 2  # Tons of sugar beet consumed by a dairy cow in a year
# These parameters are defined for clarity and completeness
selling_price_fbaby_cow, selling_price_dairy_cow, buyin_price_sugar_beet_seed, buyin_price_grain_seed, sellout_price_sugar_beet, sellout_price_grain, Acre1, B, G1, SB1

(3000, 4000, 350, 150, 80, 300, 300, 12000, 3, 2)

In [ ]:
c = [
    -selling_price_heifer,  # x1
    -selling_price_dairy_cow,  # x2
    0,  # y1
    0,  # y2
    0,  # s1
    0,  # s2
    0.1,  # w
    0,  # E
    0,  # Gr_buy
    sellout_price_grain,  # Gr_sell
    0,  # SB_buy
    sellout_price_sugar_beet,  # SB_sell
    0.04,  # U_cost
]
A = [
    # Heifers Sold Constraint
    [1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],

    # Dairy Cows Sold Constraint
    [0, 1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0],

    # Cow Housing Constraint
    [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],

    # Grain Consumption Constraint
    [0, 0, 0, 0, 0, 0, 0, 0, 1, -1, 0, 0, 0],

    # Sugar Beet Consumption Constraint
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, 0],

    # Grain Sale Constraint
    [0, 0, 0, 0, 1, 0, 0, 0, 0, -1, 0, 0, 0],

    # Sugar Beet Sale Constraint
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, -1, 0],

    # Grain Buying Constraint (modified to <=)
    [0, 0, -G1, -G1, 0, 0, 0, 0, -1, 0, 0, 0, 0],

    # Sugar Beet Buying Constraint (modified to <=)
    [0, 0, -SB1, -SB1, 0, 0, 0, 0, 0, 0, -1, 0, 0],

    # Budget Constraint
    [0, 0, 0, 0, 0, 0, 0, 0, buyin_price_grain_seed, 0, buyin_price_sugar_beet_seed, 0, 0],
]
# Upper Bounds for Inequality Constraints (b)
b = [
    0,  # Heifers Sold Constraint
    0,  # Dairy Cows Sold Constraint
    110,  # Cow Housing Constraint
    G1,  # Grain Consumption Constraint
    SB1,  # Sugar Beet Consumption Constraint
    0,  # Grain Sale Constraint
    0,  # Sugar Beet Sale Constraint
    0,  # Grain Buying Constraint
    0,  # Sugar Beet Buying Constraint
    B,   # Budget Constraint
]

# Bounds for Decision Variables (min, max) pairs
x_bounds = [
    (5, None),  # x1: Min Number of Heifers to be Sold
    (0, None),  # x2
    (5, None),  # y1: Min Number of Heifers to be Raised
    (0, None),  # y2
    (0, None),  # s1
    (0, None),  # s2
    (0, None),  # w
    (0, None),  # E
    (0, None),  # Gr_buy
    (0, None),  # Gr_sell
    (0, None),  # SB_buy
    (0, None),  # SB_sell
    (0, None),  # U_cost
]
# Solve the Linear Programming Problem
result = linprog(c, A_ub=A, b_ub=b, bounds=x_bounds, method='highs')
result

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -212500.0
              x: [ 5.000e+00  5.000e+01 ...  0.000e+00  0.000e+00]
            nit: 0
          lower:  residual: [ 0.000e+00  5.000e+01 ...  0.000e+00
                              0.000e+00]
                 marginals: [ 1.500e+03  0.000e+00 ...  8.000e+01
                              4.000e-02]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00  0.000e+00  3.000e+00
                              2.000e+00  0.000e+00  0.000e+00  1.650e+02
                              1.100e+02  1.200e+04]
                 marginals: [-2.000e+03 -2.000e+03 -2.000e+03 -0.000e+00
    

In [ ]:
# Sensitivity Analysis: Allowable Increase for each constraint
perturb = 0.01
original_profit = -result.fun

allowable_increase = []
# Perform sensitivity analysis
for i in range(len(b)):
    b_perturbed = b.copy()
    b_perturbed[i] += perturb * b[i]
    result_perturbed = linprog(c, A_ub=A, b_ub=b_perturbed, bounds=x_bounds, method='highs')
    change_in_profit = -result_perturbed.fun - original_profit
    allowable_increase.append(change_in_profit)
allowable_increase

[0.0, 0.0, 2200.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [ ]:
allowable_decrease = []
# Perform sensitivity analysis
for i in range(len(b)):
    b_perturbed = b.copy()
    b_perturbed[i] -= perturb * b[i]
    result_perturbed = linprog(c, A_ub=A, b_ub=b_perturbed, bounds=x_bounds, method='highs')
    change_in_profit = -result_perturbed.fun - original_profit
    allowable_decrease.append(change_in_profit)

allowable_decrease
import pandas as pd
# List of constraint names
constraint_names = [
    "Female Baby Cows Sold Constraint",
    "Dairy Cows Sold Constraint",
    "Cow Housing Constraint",
    "Grain Sale Constraint",
    "Sugar Beet Sale Constraint",
    "Grain Buying Constraint",
    "Sugar Beet Buying Constraint",
    "Utility Cost Constraint",
    "Workforce Cost Constraint",
    "Equipment Cost Constraint"
]

# Create a DataFrame for the table
sensitivity_table_constraints = pd.DataFrame({
    "Constraint": constraint_names,
    "Allowable Increase": allowable_increase,
    "Allowable Decrease": allowable_decrease
})
# Display the table
sensitivity_table_constraints

,Constraint,Allowable Increase,Allowable Decrease
0,Female Baby Cows Sold Constraint,0.0,0.0
1,Dairy Cows Sold Constraint,0.0,0.0
2,Cow Housing Constraint,2200.0,-2200.0
3,Grain Sale Constraint,0.0,0.0
4,Sugar Beet Sale Constraint,0.0,0.0
5,Grain Buying Constraint,0.0,0.0
6,Sugar Beet Buying Constraint,0.0,0.0
7,Utility Cost Constraint,0.0,0.0
8,Workforce Cost Constraint,0.0,0.0
9,Equipment Cost Constraint,0.0,0.0


The allowable increase column says that we can increase the cow housing constraint by 2200 before the shadow price changes.

In [ ]:
# Sensitivity Analysis: Allowable Increase for Decision Variables
perturb_obj = 0.01
original_profit_obj = -result.fun
allowable_increase_obj = []

# Perform sensitivity analysis
for i in range(len(c)):
    # Perturb the objective coefficient by a small amount (increase)
    c_perturbed = c.copy()
    c_perturbed[i] += perturb_obj * c[i]
    result_perturbed_obj = linprog(c_perturbed, A_ub=A, b_ub=b, bounds=x_bounds, method='highs')
    change_in_profit_obj = -result_perturbed_obj.fun - original_profit_obj
    allowable_increase_obj.append(change_in_profit_obj)

allowable_increase_obj
allowable_decrease_obj = []
# Perform sensitivity analysis
for i in range(len(c)):

    c_perturbed = c.copy()
    c_perturbed[i] -= perturb_obj * c[i]
    result_perturbed_obj = linprog(c_perturbed, A_ub=A, b_ub=b, bounds=x_bounds, method='highs')
    change_in_profit_obj = -result_perturbed_obj.fun - original_profit_obj
    allowable_decrease_obj.append(change_in_profit_obj)

allowable_decrease_obj
import pandas as pd
decision_variables = [
    "x1 (Female Baby Cows Sold)",
    "x2 (Dairy Cows Sold)",
    "y1 (Female Baby Cows Raised)",
    "y2 (Dairy Cows for Milk)",
    "s1 (Grain Grown)",
    "s2 (Sugar Beet Grown)",
    "w (Workforce Cost)",
    "E (Equipment Cost)",
    "Gr_buy (Buy in Grains)",
    "Gr_sell (Sell out Grains)",
    "SB_buy (Buy in Sugar Beet)",
    "SB_sell (Sell out Sugar Beet)",
    "U_cost (Utility Cost)"
]

sensitivity_table_obj = pd.DataFrame({
    "Decision Variable": decision_variables,
    "Allowable Increase": allowable_increase_obj,
    "Allowable Decrease": allowable_decrease_obj,
})
# Display the table
sensitivity_table_obj

,Decision Variable,Allowable Increase,Allowable Decrease
0,x1 (Female Baby Cows Sold),125.0,-125.0
1,x2 (Dairy Cows Sold),2000.0,-2000.0
2,y1 (Female Baby Cows Raised),0.0,0.0
3,y2 (Dairy Cows for Milk),0.0,0.0
4,s1 (Grain Grown),0.0,0.0
5,s2 (Sugar Beet Grown),0.0,0.0
6,w (Workforce Cost),0.0,0.0
7,E (Equipment Cost),0.0,0.0
8,Gr_buy (Buy in Grains),0.0,0.0
9,Gr_sell (Sell out Grains),0.0,0.0


The profit from selling one female baby cow can increase 125 dollars or decrease by 125 dollars without changing the optimum.

The profit from selling one female dairy cow can increase 2000 dollars or decrease by 2000 dollars without changing the optimum.